In [9]:
from data_keeper import get_data_keeper

In [2]:
from testing import test_models_with_drugs

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV

In [4]:
get_data_keeper().get_possible_drugs()

[u'ETHA: Ethambutol ',
 u'ISON: Isoniazid ',
 u'RIFM: rifampicin ',
 u'RIFP: Rifapentine ',
 u'PYRA: Pyrazinamide ',
 u'STRE: Streptomycin ',
 u'CYCL: Cycloserine ',
 u'ETHI: Ethionamide/ Prothionamide ',
 u'PARA: Para-aminosalicyclic acid ',
 u'AMIK: Amikacin',
 u' Kanamycin ',
 u'CAPR: Capreomycin ',
 u'OFLO: Ofloxacin ']

In [5]:
import joblib

In [7]:
from wrappers import GridSearchCVWrapper
from wrappers import XGBoostClassifierFeatureImportances as XGB
from wrappers import MatrixCleaningWrapper
from wrappers import SparseWrapper
from wrappers import ModelFeatureSelectionWrapper
from wrappers import ModelBasedFeatureImportanceGetter

In [8]:
from frn import FeatureRelevanceNetworkWrapper

In [9]:
def get_complete_linear_model():
    inner_model = LogisticRegressionCV(Cs=[10 ** i for i in xrange(-4, 4)], solver='liblinear')
    outer_model = GridSearchCV(inner_model, {'penalty': ['l1', 'l2']})
    return MatrixCleaningWrapper(SparseWrapper(outer_model))

In [10]:
def get_complete_tree_based_model():
    cv_params = {'inner_model__inner_model__n_estimators': [1],#, 5, 10, 20, 50, 100], 
                 'feature_selection_threshold_coef': [0.1]}#, 1, 3, 10, 30, 100, 300]}
    return MatrixCleaningWrapper(FeatureRelevanceNetworkWrapper(XGB(n_estimators=100), ModelBasedFeatureImportanceGetter(XGB())))

In [11]:
model = get_complete_tree_based_model()
test_models_with_drugs([('xgboost', model)], [u'ETHI: Ethionamide/ Prothionamide '], metrics=['accuracy', 'confusion matrix'])

cleaner (122, 2718)
{'f949': 1, 'f904': 1, 'f532': 9, 'f606': 10, 'f2604': 9, 'f2193': 15, 'f160': 1, 'f2607': 5, 'f1285': 1, 'f554': 2, 'f1714': 1, 'f800': 1, 'f2391': 3, 'f511': 2, 'f2056': 5, 'f2054': 7, 'f213': 1, 'f1060': 11, 'f117': 6, 'f2375': 3, 'f6': 1, 'f202': 9, 'f225': 1, 'f2100': 1, 'f900': 4, 'f652': 1, 'f2401': 1, 'f2186': 6, 'f1111': 1, 'f972': 3, 'f2177': 1, 'f970': 1, 'f458': 18, 'f2409': 8, 'f996': 30, 'f588': 3, 'f2461': 3, 'f2077': 6, 'f964': 4, 'f932': 5, 'f2469': 1, 'f2379': 1, 'f2586': 8, 'f719': 10, 'f952': 4, 'f2460': 6, 'f955': 17, 'f92': 1, 'f1917': 2, 'f2168': 16, 'f544': 1, 'f353': 3, 'f2380': 11, 'f2381': 12, 'f2040': 1, 'f1873': 3, 'f2161': 8, 'f770': 6, 'f397': 2, 'f798': 1, 'f504': 3, 'f1401': 2, 'f2363': 11, 'f316': 6, 'f211': 3, 'f210': 2, 'f1286': 1, 'f1158': 1, 'f203': 4, 'f1994': 1, 'f1884': 1, 'f2439': 15, 'f2248': 1, 'f35': 9, 'f1543': 2, 'f2356': 8, 'f2418': 8, 'f2374': 2, 'f2373': 3, 'f1937': 3, 'f2370': 8}
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.

KeyboardInterrupt: 

In [10]:
X = get_data_keeper().get_common_x()

In [11]:
from testing import results_differ_p_value

In [14]:
results_differ_p_value([0] * 100 + [1] * 32, [0] * 100 + [1] * 32, [0] * 99 + [1] * 32 + [0])

0.077071286548999973

In [12]:
from generate_subsets_for_common_x import get_ready_generator
GENERATOR_FOLDER = '/media/vlad/01D198E892261920/vlad/vlad/diploma'

In [2]:
res = get_ready_generator(folder=GENERATOR_FOLDER)
res

(<wrappers.SubsetGeneratorWrapper at 0x7fdb0e64bcb0>,
           698  1594  1771  4086  7570  7572  7581  7582  8040  9841  11370  \
 #snp_pos                                                                     
 G55413      0     0     0     1     1     0     0     0     0     0      0   
 G55414      0     1     0     1     0     0     0     0     0     0      0   
 G55415      0     0     0     0     0     0     0     0     0     0      0   
 G55416      0     0     0     1     0     0     0     0     0     0      0   
 G55417      0     0     0     0     0     0     0     0     0     0      0   
 G55418      0     0     0     1     1     0     0     0     0     0      0   
 G55419      0     0     0     0     0     0     0     0     0     0      0   
 G55421      0     1     0     1     0     0     0     1     0     0      0   
 G55422      0     0     0     0     0     0     0     0     0     0      0   
 G55423      0     0     0     0     0     0     0     0     0     0      0  

In [3]:
generator, X = res

In [4]:
generator.get_sets_count()

106625438

In [31]:
from complex_features_inserting import ExtenderStrategy, MinSizePreFilter, SimplePriorityGetter, BayesBasedPriorityGetter, ComplexFeaturesAdderWrapper
def get_simple_feature_adder_wrapper(inner_model, max_features_to_add):
    generator, matrix_before_generating = get_ready_generator()
    priority_getter = BayesBasedPriorityGetter(max_features_to_add)
    pre_filter = MinSizePreFilter(min_size=1)
    extender_strategy = ExtenderStrategy(max_features=1000, 
                                         priority_getter=priority_getter, 
                                         pre_filter=pre_filter,
                                         generator=get_ready_generator()[0])
    return ComplexFeaturesAdderWrapper(inner_model=inner_model, 
                                       matrix_before_generating=matrix_before_generating.as_matrix(),
                                       features_names=list(matrix_before_generating.columns.values),
                                       objects_names=list(matrix_before_generating.index),
                                       extender_strategy=extender_strategy)

In [32]:
from sklearn.linear_model import LogisticRegression
simple_feature_adder_wrapper = get_simple_feature_adder_wrapper(LogisticRegression(), 1000)

In [33]:
simple_feature_adder_wrapper.matrix_before_generating.shape

(142, 3175)

In [34]:
X, y = get_data_keeper().get_train_data('ETHI: Ethionamide/ Prothionamide ')

In [35]:
possible_objects = set(y.index)

In [36]:
import numpy as np
indexes = list()
y_refactored = list()

for i, el in enumerate(simple_feature_adder_wrapper.objects_names):
    if el in possible_objects:
        indexes.append(i)
        y_refactored.append(y[el])
y_refactored = np.array(y_refactored)
indexes = np.array(indexes)
indexes = indexes.reshape((indexes.shape[0], 1))

In [37]:
len(indexes)

136

In [38]:
get_ready_generator()[0].get_sets_count()

106625438

In [39]:
import sys
stdout = sys.stdout
stderr = sys.stderr

In [40]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from wrappers import XGBoostClassifierFeatureImportances as XGB
sys.stdout = sys.stderr = open("cross_val.log", "w")
cross_val_score(simple_feature_adder_wrapper, indexes, y_refactored, scoring=make_scorer(accuracy_score), cv=10)

TypeError: _get_candidate_priority() takes exactly 4 arguments (5 given)

In [ ]:
_.mean()

In [ ]:
simple_feature_adder_wrapper.extender_strategy.__dict__